In [ ]:
# importing list of drug names https://e.pcloud.link/publink/show?code=XZOMupZGQYrDo1ccMJ1P2cM1zkfSHgVWh3y

In [1]:
import pandas as pd 
import re
import sqlite3
import string
import numpy as np
import glob

In [2]:
import pandas as pd
sn = r"C:\Users\Jonathon\OneDrive\UVA\Data System Project\street_name_look_up.xlsx"
sn_df = pd.read_excel(sn)

# Display the DataFrame
print(sn_df)

        ID     QUERY                  CANDIDATE  ANNOTATION
0        1  fentanyl                  #cannabis  other drug
1        2  fentanyl  #counterfeit_prescription    non drug
2        3  fentanyl                      #drug  other drug
3        4  fentanyl                     #drugs  other drug
4        5  fentanyl                  #epidemic    non drug
...    ...       ...                        ...         ...
1087  1088      weed                        wax     synonym
1088  1089      weed                     weed..     synonym
1089  1090      weed                      weeds     synonym
1090  1091      weed                      weeed     synonym
1091  1092      weed                      xanax  other drug

[1092 rows x 4 columns]


In [3]:
sn_df = sn_df[sn_df['CANDIDATE'].str.len() >3]

In [4]:
#only using sister and synonym
sn_df = sn_df[sn_df['ANNOTATION'].isin(['synonym', 'sister'])]
sn_df

,ID,QUERY,CANDIDATE,ANNOTATION
5,6,fentanyl,#fentanyl,synonym
7,8,fentanyl,#heroin,sister
11,12,fentanyl,#opioid,sister
13,14,fentanyl,#opioids,sister
24,25,fentanyl,analogues,synonym
...,...,...,...,...
1083,1084,weed,trees,synonym
1086,1087,weed,vaped bud,synonym
1088,1089,weed,weed..,synonym
1089,1090,weed,weeds,synonym


In [5]:
# reading in more drugs by hand 
#MDMA
new_data = {'CANDIDATE': ['Disco Biscuit', 'Eve', 'Molly', 
                          'Lover’s Speed', 'STP', 'XTC', 'Uppers', 'pingers']}
#'Adam', 'X', 'E', 'Beans', 'Clarity', 'Peace',

new_df1 = pd.DataFrame(new_data)

# Add columns 'ID', 'Query', and 'Annotation' with default values
new_df1['ID'] = 0  # Replace 0 with your desired ID values
new_df1['QUERY'] = 'MDMA'
new_df1['ANNOTATION'] = 'synonym'


In [6]:
# reading in more drugs by hand 
#INHALANTS
new_data = {'CANDIDATE':  ["Dusters","Gluey", "Huff", "Laughing Gas",  "Poppers","Rush",  "Snappers",  "Whippets"]}
new_df2 = pd.DataFrame(new_data)

# Add columns 'ID', 'Query', and 'Annotation' with default values
new_df2['ID'] = 0  # Replace 0 with your desired ID values
new_df2['QUERY'] = 'INHALANTS'
new_df2['ANNOTATION'] = 'synonym'

In [7]:
# reading in more drugs by hand 
#INHALANTS
new_data = {'CANDIDATE':  ["Georgia Home Boy", "Goop", "Liquid Ecstasy", "Liquid X", ]}
new_df3 = pd.DataFrame(new_data)

# "Grievous Bodily Harm",
# Add columns 'ID', 'Query', and 'Annotation' with default values
new_df3['ID'] = 0  # Replace 0 with your desired ID values
new_df3['QUERY'] = 'GHB'
new_df3['ANNOTATION'] = 'synonym'


In [8]:
# reading in more drugs by hand 
#INHALANTS
new_data = {'CANDIDATE': ["Blow", "Bump", "Charlie", "Coke", "Snow", "Toot", "Coca", "Soda Cot", 'gak', 'marching powder']}
new_df4 = pd.DataFrame(new_data)
# removed "C",
# Add columns 'ID', 'Query', and 'Annotation' with default values
new_df4['ID'] = 0  # Replace 0 with your desired ID values
new_df4['QUERY'] = 'COCAINE'
new_df4['ANNOTATION'] = 'synonym'

In [9]:
sn_df1 = pd.concat([sn_df, new_df1,new_df2,new_df3,new_df4], ignore_index=True)

In [10]:
new_data = {'CANDIDATE': sn_df['QUERY'].unique().tolist()}
new_df5 = pd.DataFrame(new_data)

# Add columns 'ID', 'Query', and 'Annotation' with default values
new_df5['ID'] = 0  # Replace 0 with your desired ID values
new_df5['QUERY'] = sn_df['QUERY'].unique().tolist()
new_df5['ANNOTATION'] = 'synonym'
sn_df2 = pd.concat([sn_df1, new_df5], ignore_index=True)

In [11]:
marijuana_set = set(sn_df2[sn_df2['QUERY']=='marijuana']['CANDIDATE'].tolist() + sn_df2[sn_df2['QUERY']=='ganja']['CANDIDATE'].tolist() \
                     + sn_df2[sn_df2['QUERY']=='weed']['CANDIDATE'].tolist())
marijuana_list = list(marijuana_set)

In [12]:
#changing the weed / mj / ganja to a single column
sn_df2 = sn_df2[~sn_df2['QUERY'].isin(['weed','marijuana','ganja'])]
mj_temp = pd.DataFrame(marijuana_list)
mj_temp = mj_temp.rename(columns={mj_temp.columns[0]: 'CANDIDATE'})
mj_temp['QUERY'] = 'marijuana'
mj_temp['ID'] = 0
mj_temp['ANNOTATION'] = 'synonym'
mj_temp = mj_temp[sn_df2.columns.tolist()]
sn_df2 = pd.concat([sn_df2, mj_temp], ignore_index=True)
sn_df2['QUERY'].unique()


array(['fentanyl', 'heroin', 'oxy', 'MDMA', 'INHALANTS', 'GHB', 'COCAINE',
       'marijuana'], dtype=object)

In [13]:

def pre_process_text(csv,lookup,save_folder):
    #read in data 
    #file_path_submission = r"C:\Users\Jonathon\OneDrive\UVA\Data System Project\reddit_csv\submission\reddit_submission_2023_09.csv"

    #creating names for csv
    file_path_submission = csv
    long_name = file_path_submission.split("\\")[-1]
    short_name = 'clean_'+ long_name[:len(long_name)-4]

    submissions = pd.read_csv(file_path_submission)
    # removing punc & removing nulls 
    submissions['selftext'] = submissions['selftext'].fillna(' ')
    submissions['title'] = submissions['title'].fillna(' ')

    submissions['selftext_NO_PUNC'] = submissions['selftext'].str.replace("?", ' ').str.replace("!", ' ').str.replace('.', ' ').str.replace('(', ' ').str.replace(')', ' ').str.replace('"', ' ').str.replace(',', ' ')
    submissions['title_NO_PUNC'] = submissions['title'].str.replace("?", ' ').str.replace("!", ' ').str.replace('.', ' ').str.replace('(', ' ').str.replace(')', ' ').str.replace('"', ' ').str.replace(',', ' ')
    submissions['selftext_NO_PUNC'] = submissions['selftext_NO_PUNC'].str.replace('\s+', ' ').str.strip()
    submissions['title_NO_PUNC'] = submissions['title_NO_PUNC'].str.replace('\s+', ' ').str.strip()
    #SQL
    conn = sqlite3.connect('example9.db')
    
    # Use the to_sql method to write the DataFrame to a SQLite table
    submissions.to_sql('submissions', conn, index=False, if_exists='replace')
    lookup.to_sql('sn2', conn, index=False, if_exists='replace')
    # SQL query with a text-based join using LIKE
    
    match_df2 = submissions['id']
    list = lookup['QUERY'].unique()
    for x in list:
        print(x)
        query = '''
            SELECT DISTINCT 
        id,
        GROUP_CONCAT(TRIM({}), ', ') AS {},
        SUM(count_{}) AS count_{}
    FROM
        (
            SELECT DISTINCT
                a.id,
                b.CANDIDATE AS {},
                CASE WHEN upper(B.CANDIDATE) IS NULL THEN 0 ELSE count(*) END AS count_{}
            FROM
                submissions AS a
                LEFT JOIN (SELECT DISTINCT * FROM sn2 WHERE QUERY = '{}') AS b
                ON ' ' || UPPER(a.selftext_NO_PUNC) || ' ' LIKE '% ' || UPPER(b.candidate) || ' %'
                    OR ' ' || UPPER(a.title_NO_PUNC) || ' ' LIKE '% ' || UPPER(b.candidate) || ' %'
            GROUP BY a.id, b.CANDIDATE
        )
    GROUP BY id;
    
        '''.format(x, x, x, x, x, x, x ,x)
       
        result = pd.read_sql_query(query, conn)
        match_df2 = pd.merge(match_df2, result, how='left', on = 'id')
    
        #result_df = pd.concat([result_df, result], ignore_index=True)
        
    match_submissions = pd.merge(submissions,match_df2, how='left', on='id')
    
    count_columns = match_submissions.filter(like='count_')
    # Sum across the selected columns
    total_count = count_columns.sum(axis=1)
    # Add the total_count as a new column in the DataFrame
    match_submissions['total_flags'] = total_count

    match_submissions['match'] = 1  # Initialize 'no_match' column with 0
    match_submissions.loc[match_submissions['total_flags'] == 0, 'match'] = 0

    match_submissions.to_csv(save_folder+'{}.csv'.format(short_name))
    conn.commit()
    conn.close()
    return

#r"C:\Users\Jonathon\OneDrive\UVA\Data System Project\reddit_csv\submission\cleaned_matched\"

In [14]:
# loop through all submissions:
folder = r'C:\Users\Jonathon\OneDrive\UVA\Data System Project\reddit_csv\submission2' 
for x in glob.glob('{}\*.csv'.format(folder)):
    pre_process_text(x ,sn_df2 , 'C:/Users/Jonathon/OneDrive/UVA/Data System Project/reddit_csv/submission2/submissions_clean/')
# here is the submissions code


fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
fentanyl
heroin
oxy
MDMA
INHALANTS
GHB
COCAINE
marijuana
